In [1]:
from typing import Sequence, Union, Literal

import clingo

In [2]:
def solve(programs: Sequence[str],
          grounding_context=None,
          sep: str = ' '):
    ctl = clingo.Control(("--models", "0"))
    for program in programs:
        ctl.add("base", (), program)

    ctl.ground((("base", ()),), grounding_context)

    models = []

    with ctl.solve(yield_=True) as solve_handle:
        for i, model in enumerate(solve_handle):
            assert isinstance(model, clingo.Model)
            symbols = model.symbols(shown=True)
            print("Answer {}: {}{}{}{}{}".format(i + 1, "{", sep, sep.join(map(str, sorted(symbols))), sep, "}"))
            models.append(symbols)
        mode = "UNKNOWN"
        solve_result = solve_handle.get()
        if solve_result.satisfiable:
            mode = "SAT"
        elif solve_result.unsatisfiable:
            mode = "UNSAT"
        cardinality_suffix = ""
        if not solve_result.exhausted:
            cardinality_suffix = "+"
        print(mode, "{}{}".format(len(models), cardinality_suffix))

    return models

In [3]:
commodity = """

commodity("EUR").
commodity("Good").

"""
solve([commodity]);

Answer 1: { commodity("EUR") commodity("Good") }
SAT 1


In [4]:
accounts = """

account(Account, Commodity) :- register(account(Account, Commodity)).

"""
solve([accounts]);

Answer 1: {  }
SAT 1


<block>:3:32-69: info: atom does not occur in any rule head:
  register(account(Account,Commodity))



In [5]:
transaction = """

occ_at(transaction(DebitAcc, CreditAcc, Amount, Commodity), T) :- register(transaction(DebitAcc, CreditAcc, Amount, Commodity, T)).

"""
solve([transaction]);

Answer 1: {  }
SAT 1


<block>:3:67-131: info: atom does not occur in any rule head:
  register(transaction(DebitAcc,CreditAcc,Amount,Commodity,T))



In [6]:
balance = """

obs_at(balance(Account, Debits, Credits, Commodity), T) :-
  time(T),
  account(Account, Commodity),
  Debits = @debit_balance_at(Account, T),
  Credits = @credit_balance_at(Account, T).

"""
solve([balance]);

Answer 1: {  }
SAT 1


<block>:4:3-10: info: atom does not occur in any rule head:
  time(T)

<block>:5:3-30: info: atom does not occur in any rule head:
  account(Account,Commodity)



In [7]:
instance = """

time(0..3).

register(@account("Alice:Kassa", "EUR")).
register(@account("Alice:Lager", "Good")).
register(@account("Alice:Eigenkapital:EUR", "EUR")).

register(@account("Bob:Kassa", "EUR")).
register(@account("Bob:Lager", "Good")).
register(@account("Bob:Eigenkapital:Good", "Good")).

register(@transaction("Alice:Kassa", "Alice:Eigenkapital:EUR", 100, "EUR", 0)).
register(@transaction("Bob:Lager", "Bob:Eigenkapital:Good", 1, "Good", 0)).
register(@transaction("Bob:Kassa", "Alice:Kassa", 50, "EUR", 1)).
register(@transaction("Alice:Lager", "Bob:Lager", 1, "Good", 2)).
register(@transaction("Bob:Kassa", "Alice:Kassa", 50, "EUR", 3)).

"""
solve([instance]);

Answer 1: { time(0) time(1) time(2) time(3) }
SAT 1


<block>:17:10-64: info: operation undefined:
  function 'transaction' not found

<block>:16:10-64: info: operation undefined:
  function 'transaction' not found

<block>:15:10-64: info: operation undefined:
  function 'transaction' not found

<block>:14:10-74: info: operation undefined:
  function 'transaction' not found

<block>:13:10-78: info: operation undefined:
  function 'transaction' not found

<block>:11:10-51: info: operation undefined:
  function 'account' not found

<block>:10:10-39: info: operation undefined:
  function 'account' not found

<block>:9:10-38: info: operation undefined:
  function 'account' not found

<block>:7:10-51: info: operation undefined:
  function 'account' not found

<block>:6:10-41: info: operation undefined:
  function 'account' not found

<block>:5:10-40: info: operation undefined:
  function 'account' not found



In [8]:
class AccountingContext:

    def __init__(self, ledger=None):
        self.ledger = ledger
        if ledger is None:
            self.ledger = {}

    def __repr__(self):
        return repr(self.ledger)

    def __str__(self):
        return str(self.ledger)

    def __ensure_account(self, name:Union[str,int], commodity:Union[str, int, None] = None):
        if name in self.ledger:
            if commodity is not None:
                if 'commodity' in self.ledger[name]:
                    if self.ledger[name]['commodity'] != commodity:
                        raise ValueError("{} already exists with {} as commodity, here instantiated with {} as commodity", name, self.ledger['commodity'],
                           commodity)
                else:
                    self.ledger['commodity'] = commodity
        else:
            self.ledger[name] = {
                "journal": {},
                "balance sheet": {},
            }
            if commodity is not None:
                self.ledger[name]['commodity'] = commodity

    def account(self, name: clingo.Symbol, commodity: clingo.Symbol):
        name_pyrepr = AccountingContext.get_pyrepr(name)
        commodity_pyrepr = AccountingContext.get_pyrepr(commodity)
        self.__ensure_account(name_pyrepr, commodity_pyrepr)
        return clingo.Function("account", (name, commodity))

    def transaction(self,
                    debitAcc: clingo.Symbol,
                    creditAcc: clingo.Symbol,
                    amount: clingo.Symbol,
                    commodity: clingo.Symbol,
                    time: clingo.Symbol):
        debitAcc_pyrepr = AccountingContext.get_pyrepr(debitAcc)
        creditAcc_pyrepr = AccountingContext.get_pyrepr(creditAcc)
        commodity_pyrepr = AccountingContext.get_pyrepr(commodity)
        self.__ensure_account(debitAcc_pyrepr, commodity_pyrepr)
        self.__ensure_account(creditAcc_pyrepr, commodity_pyrepr)


        if time.type != clingo.SymbolType.Number:
            raise ValueError("{} is not of type clingo.{}".format(time, clingo.SymbolType.Number))
        time_pyrepr:int = time.number
        assert isinstance(time_pyrepr, int), "{} should be an int but is {}".format(time_pyrepr, type(time_pyrepr).__name__)
        if amount.type != clingo.SymbolType.Number:
            raise ValueError("{} is not of type clingo.{}".format(amount, clingo.SymbolType.Number))
        amount_pyrepr: int = amount.number
        if amount_pyrepr < 0:
            raise ValueError("{} as amount is not allowed, only use positive numbers", amount_pyrepr)

        if time_pyrepr not in self.ledger[debitAcc_pyrepr]['journal']:
            self.ledger[debitAcc_pyrepr]['journal'][time_pyrepr] = dict(debits=0, credits=0)
        if time_pyrepr not in self.ledger[creditAcc_pyrepr]['journal']:
            self.ledger[creditAcc_pyrepr]['journal'][time_pyrepr] = dict(debits=0, credits=0)

        self.ledger[debitAcc_pyrepr]['journal'][time_pyrepr]['debits'] += amount_pyrepr
        self.ledger[creditAcc_pyrepr]['journal'][time_pyrepr]['credits'] += amount_pyrepr

        return clingo.Function("transaction", (debitAcc, creditAcc, amount, commodity, time))

    def __balance_at(self, acc_pyrepr:Union[str, int], time_pyrepr:int, acc_entry:Literal['debit', 'credit']):
        acc_entries = acc_entry + 's'
        assert 'balance sheet' in self.ledger[acc_pyrepr]
        if time_pyrepr not in self.ledger[acc_pyrepr]['balance sheet']:
            self.ledger[acc_pyrepr]['balance sheet'][time_pyrepr] = {}
        if acc_entries not in self.ledger[acc_pyrepr]['balance sheet']:
            assert 'journal' in self.ledger[acc_pyrepr]
            aggregate = sum(entry[acc_entry] for t,entry in self.ledger[acc_pyrepr]['journal'].items() if t < time_pyrepr)
            self.ledger[acc_pyrepr]['balance sheet'][acc_entries] = aggregate

    def debit_balance_at(self, acc:clingo.Symbol, time:clingo.Symbol):
        if time.type != clingo.SymbolType.Number:
            raise ValueError("{} is not a clingo.{} but a clingo.{}".format(time, clingo.SymbolType.Number, time.type))
        time_pyrepr:int = time.number
        assert isinstance(time_pyrepr, int)
        acc_pyrepr = AccountingContext.get_pyrepr(acc)
        self.__ensure_account(acc_pyrepr)
        self.__balance_at(acc_pyrepr, time_pyrepr, 'debit')
        return clingo.Number(self.ledger[acc_pyrepr]['balance sheet']['debits'])

    def credit_balance_at(self, acc:clingo.Symbol, time:clingo.Symbol):
        if time.type != clingo.SymbolType.Number:
            raise ValueError("{} is not a clingo.{} but a clingo.{}".format(time, clingo.SymbolType.Number, time.type))
        time_pyrepr:int = time.number
        assert isinstance(time_pyrepr, int)
        acc_pyrepr = AccountingContext.get_pyrepr(acc)
        self.__ensure_account(acc_pyrepr)
        self.__balance_at(acc_pyrepr, time_pyrepr, 'credit')
        return clingo.Number(self.ledger[acc_pyrepr]['balance sheet']['credits'])

    @staticmethod
    def get_pyrepr(sym: clingo.Symbol) -> Union[int, str]:
        if sym.type == clingo.SymbolType.String:
            return sym.string
        elif sym.type == clingo.SymbolType.Number:
            return sym.number
        return str(sym)



In [9]:
ac = AccountingContext()
ac

{}

In [10]:
solve([accounts,
       balance,
       transaction,
       instance,
       "#show occ_at/2. #show obs_at/2."], grounding_context=ac, sep='\n');

Answer 1: {
obs_at(balance("Alice:Eigenkapital:EUR",0,0,"EUR"),0)
obs_at(balance("Alice:Eigenkapital:EUR",0,0,"EUR"),1)
obs_at(balance("Alice:Eigenkapital:EUR",0,0,"EUR"),2)
obs_at(balance("Alice:Eigenkapital:EUR",0,0,"EUR"),3)
obs_at(balance("Alice:Kassa",0,0,"EUR"),0)
obs_at(balance("Alice:Kassa",0,0,"EUR"),1)
obs_at(balance("Alice:Kassa",0,0,"EUR"),2)
obs_at(balance("Alice:Kassa",0,0,"EUR"),3)
obs_at(balance("Alice:Lager",0,0,"Good"),0)
obs_at(balance("Alice:Lager",0,0,"Good"),1)
obs_at(balance("Alice:Lager",0,0,"Good"),2)
obs_at(balance("Alice:Lager",0,0,"Good"),3)
obs_at(balance("Bob:Eigenkapital:Good",0,0,"Good"),0)
obs_at(balance("Bob:Eigenkapital:Good",0,0,"Good"),1)
obs_at(balance("Bob:Eigenkapital:Good",0,0,"Good"),2)
obs_at(balance("Bob:Eigenkapital:Good",0,0,"Good"),3)
obs_at(balance("Bob:Kassa",0,0,"EUR"),0)
obs_at(balance("Bob:Kassa",0,0,"EUR"),1)
obs_at(balance("Bob:Kassa",0,0,"EUR"),2)
obs_at(balance("Bob:Kassa",0,0,"EUR"),3)
obs_at(balance("Bob:Lager",0,0,"Good"),0)
ob

In [11]:
ac

{'Bob:Kassa': {'journal': {3: {'debits': 50, 'credits': 0}, 1: {'debits': 50, 'credits': 0}}, 'balance sheet': {0: {}, 'credits': 0, 'debits': 0, 1: {}, 2: {}, 3: {}}, 'commodity': 'EUR'}, 'Alice:Kassa': {'journal': {3: {'debits': 0, 'credits': 50}, 1: {'debits': 0, 'credits': 50}, 0: {'debits': 100, 'credits': 0}}, 'balance sheet': {0: {}, 'credits': 0, 'debits': 0, 1: {}, 2: {}, 3: {}}, 'commodity': 'EUR'}, 'Alice:Lager': {'journal': {2: {'debits': 1, 'credits': 0}}, 'balance sheet': {0: {}, 'credits': 0, 'debits': 0, 1: {}, 2: {}, 3: {}}, 'commodity': 'Good'}, 'Bob:Lager': {'journal': {2: {'debits': 0, 'credits': 1}, 0: {'debits': 1, 'credits': 0}}, 'balance sheet': {0: {}, 'credits': 0, 'debits': 0, 1: {}, 2: {}, 3: {}}, 'commodity': 'Good'}, 'Bob:Eigenkapital:Good': {'journal': {0: {'debits': 0, 'credits': 1}}, 'balance sheet': {0: {}, 'credits': 0, 'debits': 0, 1: {}, 2: {}, 3: {}}, 'commodity': 'Good'}, 'Alice:Eigenkapital:EUR': {'journal': {0: {'debits': 0, 'credits': 100}}, 'b

Does not work because there is no strict grounding order. Idea: Ground by parts.